# Load required libraries

In [1]:
import boto3
import sagemaker
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.session import Session

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# Download and Open the model output

In [2]:
import tarfile # Used to open the tar file
import pickle as pkl # Used to load the artifact
!pip install xgboost

# download the model artifact from AWS S3
!aws s3 cp s3://ads508-team4-xgboost/models/xgboost/sagemaker-xgboost-2022-04-02-20-14-08-619/output/model.tar.gz .

#opens the downloaded model artifcat and loads it as 'model' variable
tar = tarfile.open('model.tar.gz')
tar.extractall()
tar.close()
model = pkl.load(open('xgboost-model', 'rb'))

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
download: s3://ads508-team4-xgboost/models/xgboost/sagemaker-xgboost-2022-04-02-20-14-08-619/output/model.tar.gz to ./model.tar.gz


# Download the test data

In [3]:
!aws s3 cp 's3://ads508-team4-split/df_test.csv' ./data/
!aws s3 cp 's3://ads508-team4-split/df_train.csv' ./data/

download: s3://ads508-team4-split/df_test.csv to data/df_test.csv
download: s3://ads508-team4-split/df_train.csv to data/df_train.csv


In [4]:
import csv
import pandas as pd
import numpy as np

df_train = pd.read_csv(
    "./data/df_train.csv",
    delimiter=",",
    quoting=csv.QUOTE_NONE,
)


df_test = pd.read_csv(
    "./data/df_test.csv",
    delimiter=",",
    quoting=csv.QUOTE_NONE,
)

df_train = df_train.iloc[:,2:]
df_test = df_test.iloc[:,2:]


In [5]:
df_train.head()

,level_2.1,minutes_viewed,showtype_TV,genre_Adult Romance,genre_Animation,genre_Anime,genre_Comedy,genre_Crime and Mystery,genre_Documentary and Biography,genre_Drama,...,source_language_Swahili,source_language_Tagalog,source_language_Tajik,source_language_Tamil,source_language_Thai,source_language_Turkish,source_language_Urdu,source_language_Vietnamese,platform_type_mobile_phone,platform_type_web_based
0,3,26,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,5,3,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,6,67,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,6,106,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
df_test.head()

,level_2.1,minutes_viewed,showtype_TV,genre_Adult Romance,genre_Animation,genre_Anime,genre_Comedy,genre_Crime and Mystery,genre_Documentary and Biography,genre_Drama,...,source_language_Swahili,source_language_Tagalog,source_language_Tajik,source_language_Tamil,source_language_Thai,source_language_Turkish,source_language_Urdu,source_language_Vietnamese,platform_type_mobile_phone,platform_type_web_based
0,2,16,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2,6,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,1,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,90,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,0,2,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


# Feature Importance

In [7]:
import matplotlib.pyplot as plt
!pip install py-xgboost
import xgboost as xgb

#map_names = dict(zip(model.feature_names, train_data[cols_input].columns))
#model.feature_names = list(map_names.values())

#plot feature importance
#fig, ax = plt.subplots(figsize=(12,12))
#xgboost.plot_importance(model, importance_type='gain', max_num_features=30, height=0.8, ax=ax, show_values = False)
#plt.title('Feature Importance')
#plt.show()

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
ERROR: Could not find a version that satisfies the requirement py-xgboost (from versions: none)
ERROR: No matching distribution found for py-xgboost
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [8]:
train_X = df_train[['level_2.1']]
train_y = df_train.drop(['level_2.1'],axis=1)

train_y = train_y.to_numpy()

In [9]:
pred = model.predict(xgb.DMatrix(train_X))
set(pred)

{1.0}

In [10]:
from sklearn.metrics import classification_report

print(classification_report(train_y, pred, target_names='level_2.1'))

ValueError: Classification metrics can't handle a mix of multiclass-multioutput and binary targets